In [1]:
import jupyter_black
from IPython.display import display, HTML
import bs4
from data.tables import TABLE_10Q_CP_0000016875_20_000032__001
from bs4 import BeautifulSoup
import pandas as pd
import re
from io import StringIO
from litellm import completion
from dotenv import load_dotenv
from rich import print as rprint, box
from rich.panel import Panel
import textwrap
from rich.table import Table

In [2]:
load_dotenv()
jupyter_black.load()

In [3]:
def get_completion(question, markdown_table, model):
    response = completion(
        model=model,
        messages=[
            {
                "content": "Answer the user query given the markdown table. "
                "Be very mindful and explicit of units and formatting. "
                "Cells have been unmerged and only rightmost value was left with a value. "
                f"### Table\n{markdown_table}",
                "role": "system",
            },
            {"content": question, "role": "user"},
        ],
    )
    # Extract the content message and token information from the response
    content_message = response["choices"][0]["message"]["content"]
    token_info = response["usage"]
    return content_message, token_info


def answer(question, markdown_table, model):
    content_message, token_info = get_completion(question, markdown_table, model)
    rprint(Panel("\n".join(textwrap.wrap(content_message, width=80))))
    rprint(Panel(str(token_info)))

In [4]:
display(HTML(TABLE_10Q_CP_0000016875_20_000032__001))

In [5]:
def unmerge_cells(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    for td in soup.find_all("td"):
        if td.has_attr("colspan"):
            for _ in range(int(td["colspan"]) - 1):
                new_td = soup.new_tag("td")
                new_td.string = "NaN"
                td.insert_before(new_td)
            td["colspan"] = 1
    return str(soup)


unmerged = unmerge_cells(TABLE_10Q_CP_0000016875_20_000032__001)
display(HTML(unmerged))

NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
,,,,,,,,,,,,
,NaN,NaN,NaN,NaN,NaN,For the three months ended September 30,NaN,NaN,NaN,NaN,NaN,For the nine months ended September 30
(in millions of Canadian dollars),NaN,NaN,2020,NaN,NaN,2019,NaN,NaN,2020,NaN,NaN,2019
Net income,$,598,,$,618,,$,"1,642",,$,"1,776",
"Net gain (loss) in foreign currency translation adjustments, net of hedging activities",NaN,16,,NaN,(8,),NaN,(18,),NaN,23,
Change in derivatives designated as cash flow hedges,NaN,3,,NaN,2,,NaN,6,,NaN,8,
Change in pension and post-retirement defined benefit plans,NaN,44,,NaN,20,,NaN,134,,NaN,61,
Other comprehensive income before income taxes,NaN,63,,NaN,14,,NaN,122,,NaN,92,
Income tax (expense) recovery on above items,NaN,(29,),NaN,3,,NaN,(16,),NaN,(41,)
Other comprehensive income (Note 7),NaN,34,,NaN,17,,NaN,106,,NaN,51,


In [7]:
df = pd.read_html(StringIO(unmerged), flavor="lxml")[0]
display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,For the three months ended September 30,NaN,NaN,NaN,NaN,NaN,For the nine months ended September 30
3,(in millions of Canadian dollars),NaN,NaN,2020,NaN,NaN,2019,NaN,NaN,2020,NaN,NaN,2019
4,Net income,$,598,NaN,$,618,NaN,$,1642,NaN,$,1776,NaN
5,Net gain (loss) in foreign currency translatio...,NaN,16,NaN,NaN,(8,),NaN,(18,),NaN,23,NaN
6,Change in derivatives designated as cash flow ...,NaN,3,NaN,NaN,2,NaN,NaN,6,NaN,NaN,8,NaN
7,Change in pension and post-retirement defined ...,NaN,44,NaN,NaN,20,NaN,NaN,134,NaN,NaN,61,NaN
8,Other comprehensive income before income taxes,NaN,63,NaN,NaN,14,NaN,NaN,122,NaN,NaN,92,NaN
9,Income tax (expense) recovery on above items,NaN,(29,),NaN,3,NaN,NaN,(16,),NaN,(41,)


In [8]:
def to_markdown_table(df):
    df = df.dropna(how="all")  # remove empty rows
    df = df.fillna("")
    markdown_table = "\n".join(df.to_markdown(index=False).split("\n")[2:])
    markdown_table = re.sub(r" +", " ", markdown_table)
    return markdown_table


markdown_table = to_markdown_table(df)
print(markdown_table)

| | | | | | | For the three months ended September 30 | | | | | | For the nine months ended September 30 |
| (in millions of Canadian dollars) | | | 2020 | | | 2019 | | | 2020 | | | 2019 |
| Net income | $ | 598 | | $ | 618 | | $ | 1642 | | $ | 1776 | |
| Net gain (loss) in foreign currency translation adjustments, net of hedging activities | | 16 | | | (8 | ) | | (18 | ) | | 23 | |
| Change in derivatives designated as cash flow hedges | | 3 | | | 2 | | | 6 | | | 8 | |
| Change in pension and post-retirement defined benefit plans | | 44 | | | 20 | | | 134 | | | 61 | |
| Other comprehensive income before income taxes | | 63 | | | 14 | | | 122 | | | 92 | |
| Income tax (expense) recovery on above items | | (29 | ) | | 3 | | | (16 | ) | | (41 | ) |
| Other comprehensive income (Note 7) | | 34 | | | 17 | | | 106 | | | 51 | |
| Comprehensive income | $ | 632 | | $ | 635 | | $ | 1748 | | $ | 1827 | |


In [9]:
answer("whats the income tax 2020 9 months before sept 30", markdown_table, "gpt-4")

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ The income tax expense for the nine months ended September 30, 2020 was $16                                     │
│ million Canadian dollars.                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ {'prompt_tokens': 382, 'completion_tokens': 23, 'total_tokens': 405}                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [10]:
ans = answer(
    "you are now financial analyst. Give concise 3 most important insights and very brief reasoning for each",
    markdown_table,
    "gpt-4",
)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 1. Decrease in Net Income: For both the three-month and nine-month periods                                      │
│ ending September 2020, there is a noticeable decrease in net income compared to                                 │
│ 2019. This shows a potential profitability issue and may be a result of reduced                                 │
│ operations or increased expenses. The reasons behind this need to be                                            │
│ investigated further.  2. Increase in Change in Pension and Post-Retirement                                     │
│ Defined Benefit Plans: There was a significant increase in the change in pension                                │
│ and post-retirement defined benefit plans from 2019 to 2020 for both the three-                                 │
│ month and nine-month periods. This could indicate that the company is investing                                 │
│ more in their employees' retirement benefits, but it could also imply increased                                 │
│ liabilities for the firm that could affect future profitability.  3. Positive                                   │
│ Comprehensive Income: Despite a decrease in net income, the comprehensive income                                │
│ for both the three-month and nine-month periods ending in September 2020 is                                     │
│ positive, which takes into account not just profitability, but also other                                       │
│ factors like foreign currency translation adjustments and changes in derivatives                                │
│ and pension plans. This suggests that the company is still doing well overall                                   │
│ despite the decrease in net income.                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ {'prompt_tokens': 386, 'completion_tokens': 224, 'total_tokens': 610}                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯